# Olympics Data Analysis & Visualization Project

## Objective
To clean, structure, analyze, and visualize historical Olympic Games data using:

- Excel (Data Cleaning)
- DuckDB (SQL Database & Analysis)
- Python (Query Execution & Validation)
- Power BI (Data Visualization & Dashboarding)

## Project Workflow

1. Data Cleaning in Excel
2. Relational Database Creation using DuckDB
3. SQL-based Exploratory Data Analysis
4. Insight Generation
5. Interactive Dashboard Creation in Power BI

# Database Setup

A permanent DuckDB database file was created:

Database Name: olympics.duckdb

All cleaned CSV files were imported using:
read_csv_auto()

Reason for using DuckDB:
- Lightweight analytical database
- Fast CSV ingestion
- SQL support inside Python
- Suitable for large datasets

In [3]:
import pandas as pd 
import numpy as np 
import duckdb as db
connection = db.connect('olympics.duckdb')

connection.execute("""
CREATE TABLE city AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/city_clean.csv');

CREATE TABLE games AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/games_clean.csv');

CREATE TABLE sport AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/sport_clean.csv');

CREATE TABLE event AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/event_clean.csv');

CREATE TABLE person AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/person_clean.csv');

CREATE TABLE medal AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/medal_clean.csv');

CREATE TABLE competitor_event AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/competitor_event_clean.csv');

CREATE TABLE person_region AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/person_region_clean.csv');

CREATE TABLE games_competitor AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/games_competitor_clean.csv');

CREATE TABLE noc_region AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/noc_region_clean.csv');

CREATE TABLE games_city AS
SELECT * FROM read_csv_auto('C:/Hanish/sports-analysis/processed data/games_city_clean.csv');
""")

CatalogException: Catalog Error: Table with name "city" already exists!

# Data Validation

After importing:

- Verified table creation using SHOW TABLES
- Checked row counts for large tables
- Tested relational joins between tables
- Confirmed foreign key relationships logically align

Database ready for analysis.

In [4]:
connection.execute("SHOW TABLES").fetchall()

[('city',),
 ('competitor_event',),
 ('event',),
 ('games',),
 ('games_city',),
 ('games_competitor',),
 ('medal',),
 ('noc_region',),
 ('person',),
 ('person_region',),
 ('sport',)]

# Relational Model Overview

The Olympic database follows a structured relational design:

Key Relationships:

- competitor_event.medal_id → medal.id
- games_competitor.person_id → person.id
- person_region.region_id → noc_region.id
- event.sport_id → sport.id
- games_city.games_id → games.id

These relationships allow multi-table joins for advanced analysis.

In [5]:
connection.execute("""
SELECT COUNT(*) 
FROM competitor_event ce
JOIN medal m ON ce.medal_id = m.id
""").fetchall()

[(259677,)]

# Block 1: Hosting & Olympic Games Trends

## Objective
To analyze hosting patterns, city frequency, and overall Olympic Games trends over time.

This section answers:
- Are there trends in hosting frequency?
- Which cities hosted multiple times?
- How has the scale of Olympics changed?

In [ ]:
# Question 1 : Are there trends in frequency of hosting Olympic Games?

connection.execute("""
SELECT g.games_year,
       c.city_name
FROM games g
JOIN games_city gc ON g.id = gc.games_id
JOIN city c ON gc.city_id = c.id
ORDER BY g.games_year;
""").df()


,games_year,city_name
0,1896,Athina
1,1900,Paris
2,1904,St. Louis
3,1906,Athina
4,1908,London
5,1912,Stockholm
6,1920,Antwerpen
7,1924,Chamonix
8,1924,Paris
9,1928,Amsterdam


In [ ]:
# Q2 Which cities hosted olympics multiple times 

connection.execute("""
SELECT c.city_name,
       COUNT(*) AS times_hosted
FROM games_city gc
JOIN city c ON gc.city_id = c.id
GROUP BY c.city_name
HAVING COUNT(*) > 1
ORDER BY times_hosted DESC;
""").df()

,city_name,times_hosted
0,London,3
1,Athina,3
2,Los Angeles,2
3,Innsbruck,2
4,Stockholm,2
5,Sankt Moritz,2
6,Lake Placid,2
7,Paris,2


In [15]:
# Question 3: How has the scale of the olympic games changed over time ?
connection.execute("""
SELECT 
    g.games_year,
    COUNT(DISTINCT gc.person_id) AS total_participants
FROM games_competitor gc
JOIN games g ON gc.games_id = g.id
GROUP BY g.games_year
ORDER BY g.games_year Desc
limit 10;
""").df()


,games_year,total_participants
0,2016,11176
1,2014,2743
2,2012,10512
3,2010,2532
4,2008,10895
5,2006,2493
6,2004,10554
7,2002,2395
8,2000,10641
9,1998,2175


# Block 2: Sports & Events Evolution

This section analyzes how Olympic sports and events have evolved over time, 
including emerging sports, discontinued events, and popularity trends.

In [16]:
# Question 4: Emerging Sports in recent oplympics editions

connection.execute("""
SELECT 
    s.sport_name,
    MIN(g.games_year) AS first_appearance
FROM competitor_event ce
JOIN event e ON ce.event_id = e.id
JOIN sport s ON e.sport_id = s.id
JOIN games_competitor gc ON ce.competitor_id = gc.id
JOIN games g ON gc.games_id = g.id
GROUP BY s.sport_name
ORDER BY first_appearance DESC
LIMIT 10;
""").df()

,sport_name,first_appearance
0,Rugby Sevens,2016
1,Taekwondo,2000
2,Trampolining,2000
3,Triathlon,2000
4,Snowboarding,1998
5,Beach Volleyball,1996
6,Softball,1996
7,Short Track Speed Skating,1992
8,Freestyle Skiing,1992
9,Baseball,1992


In [17]:
# Question 5: Popularity Trend of Sports Over Time

connection.execute("""
SELECT 
    g.games_year,
    s.sport_name,
    COUNT(*) AS participation_count
FROM competitor_event ce
JOIN event e ON ce.event_id = e.id
JOIN sport s ON e.sport_id = s.id
JOIN games_competitor gc ON ce.competitor_id = gc.id
JOIN games g ON gc.games_id = g.id
GROUP BY g.games_year, s.sport_name
ORDER BY g.games_year, participation_count DESC;
""").df()

,games_year,sport_name,participation_count
0,1896,Gymnastics,73
1,1896,Athletics,69
2,1896,Shooting,19
3,1896,Cycling,19
4,1896,Tennis,15
...,...,...,...
901,2016,Triathlon,110
902,2016,Rhythmic Gymnastics,96
903,2016,Beach Volleyball,96
904,2016,Modern Pentathlon,72


In [18]:
# Question 6: Region-Specific Sports Dominance 

connection.execute("""
SELECT 
    s.sport_name,
    nr.region_name,
    COUNT(*) AS medal_count
FROM competitor_event ce
JOIN medal m ON ce.medal_id = m.id
JOIN event e ON ce.event_id = e.id
JOIN sport s ON e.sport_id = s.id
JOIN games_competitor gc ON ce.competitor_id = gc.id
JOIN person_region pr ON gc.person_id = pr.person_id
JOIN noc_region nr ON pr.region_id = nr.id
WHERE m.medal_name != 'NA'
GROUP BY s.sport_name, nr.region_name
ORDER BY medal_count DESC
LIMIT 20;
""").df()

,sport_name,region_name,medal_count
0,Swimming,USA,1038
1,Athletics,USA,1035
2,Swimming,Australia,402
3,Fencing,Italy,358
4,Rowing,USA,348
5,Ice Hockey,Canada,347
6,Athletics,UK,337
7,Basketball,USA,331
8,Fencing,France,307
9,Gymnastics,Soviet Union,290


In [19]:
# Question 7: Gender Imbalance in Spots Events 

connection.execute("""
SELECT 
    s.sport_name,
    SUM(CASE WHEN e.event_name LIKE '%Men%' THEN 1 ELSE 0 END) AS men_events,
    SUM(CASE WHEN e.event_name LIKE '%Women%' THEN 1 ELSE 0 END) AS women_events
FROM event e
JOIN sport s ON e.sport_id = s.id
GROUP BY s.sport_name
ORDER BY ABS(men_events - women_events) DESC;
""").df()

,sport_name,men_events,women_events
0,Shooting,68.0,7.0
1,Athletics,56.0,27.0
2,Cycling,32.0,12.0
3,Swimming,36.0,18.0
4,Wrestling,24.0,6.0
...,...,...,...
61,Table Tennis,3.0,3.0
62,Freestyle Skiing,5.0,5.0
63,Alpinism,0.0,0.0
64,Badminton,2.0,2.0


In [20]:
# Question 8: Newly Introduced Events in Recent Olympic editions 

connection.execute("""
SELECT 
    e.event_name,
    MIN(g.games_year) AS first_appearance
FROM competitor_event ce
JOIN event e ON ce.event_id = e.id
JOIN games_competitor gc ON ce.competitor_id = gc.id
JOIN games g ON gc.games_id = g.id
GROUP BY e.event_name
ORDER BY first_appearance DESC
LIMIT 20;
""").df()

,event_name,first_appearance
0,"Wrestling Women's Light-Heavyweight, Freestyle",2016
1,Rugby Sevens Women's Rugby Sevens,2016
2,Rugby Sevens Men's Rugby Sevens,2016
3,Sailing Women's Skiff,2016
4,"Wrestling Women's Featherweight, Freestyle",2016
5,Freestyle Skiing Women's Slopestyle,2014
6,Freestyle Skiing Men's Slopestyle,2014
7,Snowboarding Women's Slopestyle,2014
8,Snowboarding Men's Slopestyle,2014
9,Snowboarding Women's Parallel Slalom,2014


In [21]:
# Question 9: Discontinued or Removed Events 

connection.execute("""
SELECT 
    e.event_name,
    MAX(g.games_year) AS last_appearance
FROM competitor_event ce
JOIN event e ON ce.event_id = e.id
JOIN games_competitor gc ON ce.competitor_id = gc.id
JOIN games g ON gc.games_id = g.id
GROUP BY e.event_name
ORDER BY last_appearance ASC
LIMIT 20;
""").df()

,event_name,last_appearance
0,"Shooting Men's Military Rifle, 200 metres",1896
1,"Cycling Men's 10,000 metres",1896
2,"Wrestling Men's Unlimited Class, Greco-Roman",1896
3,Cycling Men's 12-Hours Race,1896
4,"Gymnastics Men's Horizontal Bar, Teams",1896
5,"Gymnastics Men's Parallel Bars, Teams",1896
6,"Swimming Men's 1,200 metres Freestyle",1896
7,"Shooting Men's Military Pistol, 25 metres",1896
8,"Shooting Men's Muzzle-Loading Pistol, 25 metres",1896
9,Swimming Men's 500 metres Freestyle,1896


# Block 3: Athlete & Performance Trends 

In [22]:
# Question 10: Height and Weight Trends Over Time 

connection.execute("""
SELECT 
    g.games_year,
    ROUND(AVG(p.height), 2) AS avg_height,
    ROUND(AVG(p.weight), 2) AS avg_weight
FROM games_competitor gc
JOIN person p ON gc.person_id = p.id
JOIN games g ON gc.games_id = g.id
WHERE p.height IS NOT NULL AND p.weight IS NOT NULL
GROUP BY g.games_year
ORDER BY g.games_year;
""").df()

,games_year,avg_height,avg_weight
0,1896,174.33,64.08
1,1900,175.10,49.25
2,1904,175.77,44.72
3,1906,177.23,63.50
4,1908,176.62,57.74
5,1912,177.41,60.86
6,1920,175.82,46.37
7,1924,175.61,47.96
8,1928,175.81,52.55
9,1932,174.31,32.42


In [23]:
# Question 11: Dominant Countries in Specific Sports 

connection.execute("""
SELECT 
    s.sport_name,
    nr.region_name,
    COUNT(*) AS medal_count
FROM competitor_event ce
JOIN medal m ON ce.medal_id = m.id
JOIN event e ON ce.event_id = e.id
JOIN sport s ON e.sport_id = s.id
JOIN games_competitor gc ON ce.competitor_id = gc.id
JOIN person_region pr ON gc.person_id = pr.person_id
JOIN noc_region nr ON pr.region_id = nr.id
WHERE m.medal_name != 'NA'
GROUP BY s.sport_name, nr.region_name
ORDER BY s.sport_name, medal_count DESC;
""").df()

,sport_name,region_name,medal_count
0,Aeronautics,Switzerland,1
1,Alpine Skiing,Austria,114
2,Alpine Skiing,Switzerland,59
3,Alpine Skiing,France,45
4,Alpine Skiing,USA,41
...,...,...,...
1501,Wrestling,Tunisia,1
1502,Wrestling,Serbia and Montenegro,1
1503,Wrestling,United Arab Republic,1
1504,Wrestling,Puerto Rico,1
